In [16]:
# import modules from root of project
%cd ..

from arena.arena import parse_stimulus_matrix, Sheet, run_sheets, collect_actuation_sheets, SheetInvocation, \
    lql_to_sheet_signature
from arena.engine.adaptation import PassThroughAdaptationStrategy
from arena.engine.artifacts import CodeCandidate, import_classes_under_test
from arena.engine.classes import ClassUnderTest
from arena.engine.ssntestdriver import interpret_sheet, run_sheet, InvocationListener, Test, TestInvocation
from arena.lql.lqlparser import parse_lql
from arena.ssn.ssnparser import parse_sheet

# pandas
import pandas as pd
pd.set_option('display.max_columns', 0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', 0) #replace n with the number of rows you want to see completely

/


# Create Stimulus Matrix for Base64 Example

In [21]:
# lql (interface specification)
lql = """Base64 {
        base64_encode(str)->str
    }
"""

# stimulus sheet
ssn_jsonl = """
            {"cells": {"A1": {}, "B1": "create", "C1": "Base64"}}
            {"cells": {"A2": {}, "B2": "base64_encode", "C2": "A1", "D2": "'Hello World!'"}}
"""

# classes under test
base_folder = f"/home/marcus/PycharmProjects/arena-python/code-samples/base64" # FIXME must be changed to local directory of the project
c1 = CodeCandidate("917239ca-5093-44a6-a284-64e1acb8ccac", "Base64", f"{base_folder}/917239ca-5093-44a6-a284-64e1acb8ccac/candidate.py")
c2 = CodeCandidate("c108afda-e52c-454b-a7ed-c05f48257a9b", "Base64", f"{base_folder}/c108afda-e52c-454b-a7ed-c05f48257a9b/candidate.py")
c3 = CodeCandidate("c9571f41-161b-46ed-a528-941d96a0dd2b", "Base64", f"{base_folder}/c9571f41-161b-46ed-a528-941d96a0dd2b/candidate.py")
cuts = import_classes_under_test([c1, c2, c3])

# create stimulus matrix
sm = parse_stimulus_matrix([Sheet("test1()", ssn_jsonl, lql)], cuts, [SheetInvocation("test1", "")])

## SM is a pandas DataFrame

The data frame consists of complex objects.

In [22]:
sm

,<arena.engine.classes.ClassUnderTest object at 0x78aac8b8bb60>,<arena.engine.classes.ClassUnderTest object at 0x78aac8b89a00>,<arena.engine.classes.ClassUnderTest object at 0x78aac8b88b30>
0,<arena.engine.ssntestdriver.TestInvocation obj...,<arena.engine.ssntestdriver.TestInvocation obj...,<arena.engine.ssntestdriver.TestInvocation obj...


## Run Stimulus Matrix in Arena

In [23]:
# run stimulus matrix
invocation_listener = InvocationListener()
srm = run_sheets(sm, 1, invocation_listener)

## SRM is a pandas DataFrame

The data frame consists of complex objects.

In [24]:
srm

,<arena.engine.adaptation.AdaptedImplementation object at 0x78aad4216de0>,<arena.engine.adaptation.AdaptedImplementation object at 0x78aae91eb230>,<arena.engine.adaptation.AdaptedImplementation object at 0x78aac8b459d0>
0,Executed Invocations\n0 => ExecutedInvocation ...,Executed Invocations\n0 => ExecutedInvocation ...,Executed Invocations\n0 => ExecutedInvocation ...


## Zoom into SRM

In [25]:
# create actuation sheets, now we have the real stimulus response matrix (SRM)
srm_actuations = collect_actuation_sheets(srm)

In [26]:
srm_actuations

,<arena.engine.adaptation.AdaptedImplementation object at 0x78aad4216de0>,<arena.engine.adaptation.AdaptedImplementation object at 0x78aae91eb230>,<arena.engine.adaptation.AdaptedImplementation object at 0x78aac8b459d0>
0,output operation ...,...,output ... ...


### Get Actuation sheet for first cell in the DataFrame

In [27]:
srm_actuations.iloc[0][0] # pick first cell

/tmp/ipykernel_275390/2780891456.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  srm_actuations.iloc[0][0] # pick first cell


,output,operation,service,input_0
0,$CUT@Base64@0,__init__,OWLGYVBGPR.Base64,None
1,SGVsbG8gV29ybGQh,base64_encode,$CUT@Base64@0,Hello World!


### Which Candidate Implementation is it?

In [28]:
vars(srm_actuations.columns[0]) # print properties of object

{'cut': <arena.engine.classes.ClassUnderTest at 0x78aac8b8bb60>,
 'initializer_mapping': {},
 'method_mapping': {<arena.lql.lqlparser.MethodSignature at 0x78aac8b885c0>: <function 917239ca-5093-44a6-a284-64e1acb8ccac.Base64.base64_encode(self, data)>},
 'adapter_id': '0'}

In [29]:
#  show class
vars(srm_actuations.columns[0].cut) # print properties of object

{'id': '917239ca-5093-44a6-a284-64e1acb8ccac',
 'code_candidate': <arena.engine.artifacts.CodeCandidate at 0x78aac8b8a090>,
 'class_under_test': 917239ca-5093-44a6-a284-64e1acb8ccac.Base64}

### Which Test is it?

In [30]:
# from Stimulus Matrix
vars(sm.iloc[0][0].test) # print properties of object

/tmp/ipykernel_275390/1479926970.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vars(sm.iloc[0][0].test) # print properties of object


{'name': 'test1',
 'parsed_sheet': <arena.ssn.ssnparser.ParsedSheet at 0x78aac8b89ee0>,
 'interface_specification': <arena.lql.lqlparser.Interface at 0x78aac8b8a570>,
 'signature': <arena.arena.SheetSignature at 0x78aac8b8aa20>}